# Imports

In [104]:
import ydata_profiling
import pandas as pd
import re

# Import of the datasets

In [105]:
nba = pd.read_csv("./Datasets/nba_elo.csv")
pollution = pd.read_csv("./Datasets/pollution_2000_2023.csv", index_col=[0])
us_accidents = pd.read_csv("./Datasets/US_Accidents_March23.csv")
nba_teams = pd.read_csv("./Datasets/team.csv")

Preserving the original data to not lose any information:

In [106]:
nba_original = nba.copy()
pollution_original = pollution.copy()
us_accidents_original = us_accidents.copy()
nba_teams_original = nba_teams.copy()

Reduction of the US Traffic Accidents dataset to only the important columns.

In [107]:
us_accidents = us_accidents[["Severity", "Start_Time", "City", "State", 
              "Amenity", "Bump", "Crossing","Give_Way",
              "Junction","No_Exit","Railway","Roundabout",
              "Station","Stop","Traffic_Calming","Traffic_Signal", "Start_Lat", "Start_Lng"]]

# Profiling of the datasets

## NBA dataset

In [108]:
profile = ydata_profiling.ProfileReport(nba, title="")
profile.to_file("Datasets_profiling/nba_profiling.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 98.20it/s]


### Discussion of the data quality in the NBA dataset

In [109]:
pd.Series(nba.duplicated()).unique()

array([False])

#### Standardizing string columns

To standardize the Address, State and County columns we'll do the following:
* Make them all lower case;
* Remove all whitespace;
* Remove all spaces;
* Remove all special characters;

In [110]:
# Making all cities lower case.
nba["team1"] = nba["team1"].str.upper()
nba["team2"] = nba["team2"].str.upper()

# Removal of whitespace from all cities.
nba["team1"] = nba["team1"].str.strip()
nba["team2"] = nba["team2"].str.strip()

# Removal of spaces from cities.
nba["team1"] = nba["team1"].str.replace(" ", "")
nba["team2"] = nba["team2"].str.replace(" ", "")

# Removal of special characters.
nba["team1"] = nba["team1"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
nba["team2"] = nba["team2"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

#### Cleaning the Playoff column

The playoff column has a very strange peculiarity. \ 
Before 2016 playoff games are marked as true ('t' in the dataset ) or nothing if they're not playoff games (NaN in the dataset). \ 
However, from 2016 onwards the dataset marks the type of playoff game, instead of just using 't'. \
The following characters are used in the dataset to designate:
* 'q': Quarterfinal 
* 's': Semifinal
* 'c': Conference final
* 'f': Final

For simplicity's sake we'll consider any playoff match as just a True and the rest as False. \
However, it's worth nothing that it could prove to be interesting to analyze if a team losing a final has a bigger impact than a regular playoff game.

In [111]:
nba["playoff"].value_counts()

t    4033
q     144
s      43
c      22
f      12
Name: playoff, dtype: int64

In [112]:
nba["playoff"] = nba["playoff"].apply(lambda x: True if x == 't' or x == 'q' or x == 's' or x == 'f' or x == 'c' else False)

In [113]:
nba["playoff"].value_counts()

False    62814
True      4254
Name: playoff, dtype: int64

## Air Pollution dataset

In [114]:
profile = ydata_profiling.ProfileReport(pollution, title="")
profile.to_file("Datasets_profiling/air_pollution_profiling.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 52.86it/s]


### Discussion of the data quality in the Air Pollution dataset

* Duplicate rows: This datasate has 1739 duplicate rows (0.3%).

In [115]:
pollution.drop_duplicates(inplace=True)

We drop duplicate rows from this dataset since we've the date when it occurred it makes no sense to have duplicate rows.

#### Standardizing String columns

To standardize the Address, State and County columns we'll do the following:
* Make them all upper case;
* Remove all whitespace;
* Remove all spaces;
* Remove all special characters;

In [116]:
# Making all cities lower case.
pollution["City"] = pollution["City"].str.upper()
pollution["State"] = pollution["State"].str.upper()
pollution["County"] = pollution["County"].str.upper()

# Removal of whitespace from all cities.
pollution["City"] = pollution["City"].str.strip()
pollution["State"] = pollution["State"].str.strip()
pollution["County"] = pollution["County"].str.strip()

# Removal of spaces from cities.
pollution["City"] = pollution["City"].str.replace(" ", "")
pollution["State"] = pollution["State"].str.replace(" ", "")
pollution["County"] = pollution["County"].str.replace(" ", "")

# Removal of special characters.
pollution["City"] = pollution["City"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
pollution["State"] = pollution["State"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
pollution["County"] = pollution["County"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

#### Cleaning errors in the state column

In [117]:
import py_stringmatching.similarity_measure.levenshtein as lev

us_states = [
    "ALABAMA", "ALASKA", "ARIZONA", "ARKANSAS", "CALIFORNIA", "COLORADO",
    "CONNECTICUT", "DELAWARE", "FLORIDA", "GEORGIA", "HAWAII", "IDAHO",
    "ILLINOIS", "INDIANA", "IOWA", "KANSAS", "KENTUCKY", "LOUISIANA",
    "MAINE", "MARYLAND", "MASSACHUSETTS", "MICHIGAN", "MINNESOTA",
    "MISSISSIPPI", "MISSOURI", "MONTANA", "NEBRASKA", "NEVADA",
    "NEWHAMPSHIRE", "NEWJERSEY", "NEWMEXICO", "NEWYORK",
    "NORTHCAROLINA", "NORTHDAKOTA", "OHIO", "OKLAHOMA", "OREGON",
    "PENNSYLVANIA", "RHODEISLAND", "SOUTHCAROLINA", "SOUTHDAKOTA",
    "TENNESSEE", "TEXAS", "UTAH", "VERMONT", "VIRGINIA", "WASHINGTON",
    "WESTVIRGINIA", "WISCONSIN", "WYOMING"
]

def clean_state(df, col_name):
    leven = lev.Levenshtein()

    def best_match(state_str):
        max_score = -1
        best_state = state_str
        for state in us_states:
            score = leven.get_sim_score(state_str, state)
            if score > max_score:
                max_score = score
                best_state = state
        return best_state

    df[col_name] = df[col_name].apply(best_match)

clean_state(pollution, "State")

This code takes about ~3 minutes to execute.

### Standardizing numerical columns

To standardize the numerical columns we'll do the following procedures:
* Fill missing values with the median;
* Check for values out of bounds;

In [118]:
o3_median = pollution["O3 AQI"].median()
co_median = pollution["CO AQI"].median()
so2_median = pollution["SO2 AQI"].median()
no2_median = pollution["NO2 AQI"].median()

pollution["O3 AQI"].fillna(o3_median, inplace=True)
pollution["CO AQI"].fillna(co_median, inplace=True)
pollution["SO2 AQI"].fillna(so2_median, inplace=True)
pollution["NO2 AQI"].fillna(no2_median, inplace=True)

The AQI in the US can only have values between 0-500, therefore we need to look for values out of this range and clean them.

In [119]:
pollution["O3 AQI"] = pollution["O3 AQI"].apply(lambda x: o3_median if x < 0 or x > 500 else x)
pollution["CO AQI"] = pollution["CO AQI"].apply(lambda x: co_median if x < 0 or x > 500 else x)
pollution["SO2 AQI"] = pollution["SO2 AQI"].apply(lambda x: so2_median if x < 0 or x > 500 else x)
pollution["NO2 AQI"] = pollution["NO2 AQI"].apply(lambda x: no2_median if x < 0 or x > 500 else x)

## NBA teams dataset

In [120]:
profile = ydata_profiling.ProfileReport(nba_teams, title="")
profile.to_file("Datasets_profiling/nba_teams_profiling.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 361.80it/s]


### Discussion of the data quality in the NBA teams dataset

This dataset is very small, having only 30 rows of data. Therefore, if it had any errors they could easily be corrected by hand. \
However, we'll define some automatic cleaning routines anyway.

In [121]:
nba_teams.drop_duplicates(inplace=True)

There are no duplicate rows in this dataset, and if there are any duplciated rows inserted it makes no sense to keep them. Since these rows just represent general information about a team, having more than one row for a specific team makes no sense.

#### Standardizing String columns

To standardize the State, Nickname, Abbreviation, City and Full_Name columns we'll do the following:
* Make them all lower case;
* Remove all whitespace;
* Remove all spaces;
* Remove all special characters;

In [122]:
# Making all cities lower case.
nba_teams["city"] = nba_teams["city"].str.upper()
nba_teams["state"] = nba_teams["state"].str.upper()
nba_teams["nickname"] = nba_teams["nickname"].str.upper()
nba_teams["full_name"] = nba_teams["full_name"].str.upper()

# Removal of whitespace from all cities.
nba_teams["city"] = nba_teams["city"].str.strip()
nba_teams["state"] = nba_teams["state"].str.strip()
nba_teams["nickname"] = nba_teams["nickname"].str.strip()
nba_teams["full_name"] = nba_teams["full_name"].str.strip()

# Removal of spaces from cities.
nba_teams["city"] = nba_teams["city"].str.replace(" ", "")
nba_teams["state"] = nba_teams["state"].str.replace(" ", "")
nba_teams["nickname"] = nba_teams["nickname"].str.replace(" ", "")
nba_teams["full_name"] = nba_teams["full_name"].str.replace(" ", "")

# Removal of special characters.
nba_teams["city"] = nba_teams["city"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
nba_teams["state"] = nba_teams["state"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
nba_teams["nickname"] = nba_teams["nickname"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
nba_teams["full_name"] = nba_teams["full_name"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

#### Cleaning errors in the state column

To find errors in the state column we'll use the Levenshtein similarity measure to find typos in the state names.

In [123]:
import py_stringmatching.similarity_measure.levenshtein as lev

us_states = [
    "ALABAMA", "ALASKA", "ARIZONA", "ARKANSAS", "CALIFORNIA", "COLORADO",
    "CONNECTICUT", "DELAWARE", "FLORIDA", "GEORGIA", "HAWAII", "IDAHO",
    "ILLINOIS", "INDIANA", "IOWA", "KANSAS", "KENTUCKY", "LOUISIANA",
    "MAINE", "MARYLAND", "MASSACHUSETTS", "MICHIGAN", "MINNESOTA",
    "MISSISSIPPI", "MISSOURI", "MONTANA", "NEBRASKA", "NEVADA",
    "NEWHAMPSHIRE", "NEWJERSEY", "NEWMEXICO", "NEWYORK",
    "NORTHCAROLINA", "NORTHDAKOTA", "OHIO", "OKLAHOMA", "OREGON",
    "PENNSYLVANIA", "RHODEISLAND", "SOUTHCAROLINA", "SOUTHDAKOTA",
    "TENNESSEE", "TEXAS", "UTAH", "VERMONT", "VIRGINIA", "WASHINGTON",
    "WESTVIRGINIA", "WISCONSIN", "WYOMING"
]

clean_state(nba_teams, "state")

## US Traffic Accidents dataset

In [124]:
profile = ydata_profiling.ProfileReport(us_accidents, title="")
profile.to_file("Datasets_profiling/us_accidents_profiling.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 291.29it/s]


### Discussion of the data quality in the US Traffic Accidents dataset

* Duplicate rows: This dataset has 502007 duplicate rows (6.5%).

In [125]:
us_accidents.drop_duplicates(inplace=True)

These rows get dropped, since they are exact replicas of other rows and repition in this dataset doesn't add any valuable information.

* City columns: Has 256 NaN values and some are upper case and lower case.

In [126]:
us_accidents[us_accidents.City.isnull()]

,Severity,Start_Time,City,State,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Start_Lat,Start_Lng
85968,2,2016-08-02 18:18:02,NaN,CA,False,False,False,False,False,False,False,False,False,False,False,False,34.451862,-117.660103
111080,2,2016-06-01 10:26:08,NaN,CA,False,False,False,False,False,False,False,False,False,False,False,False,34.451862,-117.660103
119772,2,2016-12-23 19:30:59,NaN,FL,False,False,True,False,False,False,False,False,False,False,False,True,27.388653,-82.441948
122929,2,2017-01-17 17:34:09,NaN,FL,False,False,False,False,False,False,False,False,False,False,False,False,27.387951,-82.440239
123702,2,2017-01-24 07:30:44,NaN,FL,False,False,True,False,False,False,False,False,False,False,False,True,27.388653,-82.441948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7597826,2,2017-12-13 18:38:56,NaN,DC,False,False,False,False,False,False,False,False,False,False,False,True,38.965710,-77.002460
7606562,2,2017-11-06 09:08:42,NaN,NY,False,False,True,False,True,False,False,False,False,True,False,False,41.036740,-73.675490
7614480,4,2017-11-24 04:48:21,NaN,IA,False,False,False,False,False,False,False,False,False,True,False,False,40.586870,-92.985090
7619724,2,2017-10-09 15:46:08,NaN,NY,False,False,True,False,True,False,False,False,False,True,False,False,41.036740,-73.675490


#### Filling NaN city values using coordinates

#### Standardizing String columns

Using a geolocator api, we can fill more than half of the NaN city values, as for the other NaN values they get a NoCity value.

This is because they happen between cities or villages.

In [134]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geo_lookup")

idx_location_list = list()
idx_no_city = list()
for index in us_accidents[us_accidents.City.isnull()].index:

    # Obtaining the Latitude and the Longitute associated with the row.
    latitude = us_accidents.loc[index, "Start_Lat"]
    longitude = us_accidents.loc[index, "Start_Lng"]
    location = geolocator.reverse((latitude, longitude), exactly_one=True)

    # Check if it's a city, a locality or a village.
    address = location.raw.get('address', {})
    city = address.get('city', None)
    locality = address.get('locality', None)
    village = address.get('village', None)

    us_accidents.loc[index, "City"] = city if city else locality if locality else village if village else "NOCITY"


This code takes about ~4-5 minutes to execute.

To standardize the State and City columns we'll do the following:
* Make them all lower case;
* Remove all whitespace;
* Remove all spaces;
* Remove all special characters;

In [135]:
# Making all cities lower case.
us_accidents["City"] = us_accidents["City"].str.upper()
us_accidents["State"] = us_accidents["State"].str.upper()

# Removal of whitespace from all cities.
us_accidents["City"] = us_accidents["City"].str.strip()
us_accidents["State"] = us_accidents["State"].str.strip()

# Removal of spaces from cities.
us_accidents["City"] = us_accidents["City"].str.replace(" ", "")
us_accidents["State"] = us_accidents["State"].str.replace(" ", "")

# Removal of special characters.
us_accidents["City"] = us_accidents["City"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
us_accidents["State"] = us_accidents["State"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

This code takes about ~30 seconds to execute.

#### Cleaning errors in the State column

Using the Jaro similarity measure we compute the differences between state abbreviations to correct typos.

The Jaro measure is a type of edit distance, developed mainly to compare short strings, such as first and last names.

In [136]:
import py_stringmatching.similarity_measure.jaro as jaro

us_state_abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

def clean_state_abbreviations(df, col_name):
    leven = lev.Levenshtein()

    def best_match(state_str):
        max_score = -1
        best_state = state_str
        for state in us_state_abbreviations:
            score = leven.get_sim_score(state_str, state)
            if score > max_score:
                max_score = score
                best_state = state
        return best_state

    df[col_name] = df[col_name].apply(best_match)

clean_state_abbreviations(us_accidents, "State")

This code takes about ~16 minutes to run.